In [ ]:
import kagglehub
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np
import os

path = kagglehub.dataset_download("fronkongames/steam-games-dataset")

print("保存先パス:", path)

csv_path = os.path.join(path, "games.csv")
df = pd.read_csv(csv_path)

保存先パス: /Users/kavuk/.cache/kagglehub/datasets/fronkongames/steam-games-dataset/versions/31


In [6]:
print(df[['Name', 'Peak CCU', 'Price']].head())

                 Name  Peak CCU  Price
2539430   Aug 1, 2023         0      0
496350   Jul 29, 2016         0     65
1034400   May 6, 2019         0      0
3292190  Oct 31, 2024         0      0
3631080  Apr 24, 2025         0      0


In [10]:
# df = df.set_index('AppID')
df = pd.read_csv(csv_path, index_col=False, quotechar='"')
df.head()

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DiscountDLC count,About the game,Supported languages,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
0,2539430,Black Dragon Mage Playtest,"Aug 1, 2023",0 - 0,0,0,0.00,0,0,NaN,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,https://shared.akamai.steamstatic.com/store_it...
1,496350,Supipara - Chapter 1 Spring Has Come!,"Jul 29, 2016",0 - 20000,0,0,5.24,65,0,"Springtime, April: when the cherry trees come ...",...,8,0,8,0,minori,MangaGamer,"Single-player,Steam Trading Cards,Steam Cloud,...",Adventure,"Adventure,Visual Novel,Anime,Cute",https://shared.akamai.steamstatic.com/store_it...
2,1034400,Mystery Solitaire The Black Raven,"May 6, 2019",0 - 20000,0,0,4.99,0,0,"Immerse yourself in the most beloved, mystical...",...,0,0,0,0,Somer Games,8floor,"Single-player,Family Sharing",Casual,"Casual,Card Game,Solitaire,Puzzle,Hidden Objec...",https://shared.akamai.steamstatic.com/store_it...
3,3292190,버튜버 파라노이아 - Vtuber Paranoia,"Oct 31, 2024",0 - 20000,1,0,8.99,0,1,"synopsis 'Hello, I'm Hiyoro, a new YouTuber!' ...",...,0,0,0,0,유진게임즈,유진게임즈,"Single-player,Steam Achievements,Family Sharing","Casual,Indie,Simulation",NaN,https://shared.akamai.steamstatic.com/store_it...
4,3631080,Maze Quest VR,"Apr 24, 2025",0 - 20000,0,0,4.99,0,0,Its not just a Maze; its a Quest! Enter the ca...,...,0,0,0,0,Reality Expanded LLC,Reality Expanded LLC,"Single-player,VR Only,Steam Leaderboards,Famil...","Action,Early Access",NaN,https://shared.akamai.steamstatic.com/store_it...


In [11]:
print(df[['Name', 'Peak CCU', 'Price']].head())

                                    Name  Peak CCU  Price
0             Black Dragon Mage Playtest         0   0.00
1  Supipara - Chapter 1 Spring Has Come!         0   5.24
2      Mystery Solitaire The Black Raven         0   4.99
3            버튜버 파라노이아 - Vtuber Paranoia         1   8.99
4                          Maze Quest VR         0   4.99


In [ ]:
df['log_ccu'] = np.log(df['Peak CCU'] + 1)
zero_count = (df['Peak CCU'] == 0).sum()
print(f"同時接続ゼロのゲーム数: {zero_count} / 全体: {len(df)}")

print(f"log_ccu の最大値: {df['log_ccu'].max()}")

同時接続ゼロのゲーム数: 102935 / 全体: 122611
log_ccu の最大値: 13.829351331025592
